In [39]:
import pandas as pd
import torch
from Network import *

# 选取需要的列
COLUMN_INDEX = [
    "Temp (°C)",
    "Dew Point Temp (°C)",
    "Rel Hum (%)",
    "Wind Spd (km/h)",
    "Stn Press (kPa)",
]
data = pd.read_csv("Data/DataSet.csv")[COLUMN_INDEX].to_numpy()
data_tensor = torch.tensor(data, dtype=torch.float32)

data_tensor

tensor([[  1.5000,  -3.6000,  69.0000,  39.0000,  99.8100],
        [  1.5000,  -3.9000,  67.0000,  35.0000, 100.0100],
        [  1.0000,  -4.3000,  68.0000,  32.0000, 100.1400],
        ...,
        [  4.0000,   3.6000,  97.0000,  32.0000,  99.1500],
        [  4.0000,   3.6000,  97.0000,  30.0000,  98.8000],
        [  4.0000,   3.7000,  98.0000,  15.0000,  98.5700]])

In [67]:
RATIO=0.7
NUM_STEPS=48
BATCH_SIZE=32

def PERIOD_DATA(DATA,STARTING_INDEX,NUM_STEPS):
    return DATA[STARTING_INDEX:STARTING_INDEX+NUM_STEPS,:]

def TimeSeriesDataSplit(DATA,NUM_STEPS=None,RATIO=0.8,SHUFFLE=True):
    if NUM_STEPS is None:
        NUM_STEPS=7*24
    if SHUFFLE:
        INDEX=torch.randperm(DATA.shape[0]-NUM_STEPS)
    else:
        INDEX=torch.arange(0,DATA.shape[0]-NUM_STEPS)    
    
    TRAIN_SET_INDEX=INDEX[:int(RATIO*len(INDEX))]
    TEST_SET_INDEX=INDEX[int(RATIO*len(INDEX)):]
    TRAIN_SET_DATA=[PERIOD_DATA(DATA,index,NUM_STEPS) for index in TRAIN_SET_INDEX]
    TEST_SET_DATA=[PERIOD_DATA(DATA,index,NUM_STEPS) for index in TEST_SET_INDEX]

    return TRAIN_SET_DATA,TEST_SET_DATA,TRAIN_SET_INDEX,TEST_SET_INDEX

def Data2Loader(DATA,INDEX,BATCH_SIZE,NUM_STEPS):
    # Convert dataset to Dataloaders. Dataset elements are DATA[0].shape[1]
    # \times NUM_STEPS shaped tensors
    NUM_SUBSEQUENCES=len(DATA)
    NUM_BATCHES=NUM_SUBSEQUENCES//BATCH_SIZE
    for i in range(NUM_BATCHES):
        yield PERIOD_DATA(DATA,INDEX[i],NUM_STEPS),PERIOD_DATA(DATA,INDEX[i]+1,NUM_STEPS)
# _,_,temp1,temp2=TimeSeriesDataSplit(DATA=data_tensor,NUM_STEPS=NUM_STEPS,RATIO=0.01)
# temp3=Data2Loader(DATA=data_tensor,INDEX=temp1,BATCH_SIZE=BATCH_SIZE,NUM_STEPS=NUM_STEPS)



In [68]:
def TimeSeriesDataSplit2Loaders(DATA,BATCH_SIZE=None,NUM_STEPS=None,RATIO=0.8,SHUFFLE=True):
    if BATCH_SIZE is None:
        BATCH_SIZE=32
    if NUM_STEPS is None:
        NUM_STEPS=7*24
    if SHUFFLE:
        INDEX=torch.randperm(DATA.shape[0]-NUM_STEPS-1)
    else:
        INDEX=torch.arange(0,DATA.shape[0]-NUM_STEPS-1)
    
    TRAIN_SET_INDEX=INDEX[:int(RATIO*len(INDEX))]
    TEST_SET_INDEX=INDEX[int(RATIO*len(INDEX)):]
    TRAIN_SET_DATA_FORMMER=[PERIOD_DATA(DATA,index,NUM_STEPS) for index in TRAIN_SET_INDEX]
    TRAIN_SET_DATA_LATTER=[PERIOD_DATA(DATA,index+1,NUM_STEPS) for index in TRAIN_SET_INDEX]
    TEST_SET_DATA_FORMMER=[PERIOD_DATA(DATA,index,NUM_STEPS) for index in TEST_SET_INDEX]
    TEST_SET_DATA_LATTER=[PERIOD_DATA(DATA,index+1,NUM_STEPS) for index in TEST_SET_INDEX]


    return (DataLoader((TRAIN_SET_DATA_FORMMER,TRAIN_SET_DATA_LATTER)
                       ,batch_size=BATCH_SIZE),
            DataLoader((TEST_SET_DATA_FORMMER,TEST_SET_DATA_LATTER)
                       ,batch_size=BATCH_SIZE),
            TRAIN_SET_INDEX,TEST_SET_INDEX)

train_loader,test_loader,train_index,test_index=TimeSeriesDataSplit2Loaders(
    data_tensor,
    BATCH_SIZE=BATCH_SIZE,
    NUM_STEPS=NUM_STEPS,
    RATIO=0.5)


In [81]:
temp=25
next(iter(train_loader))[0][0,temp,:]==next(iter(train_loader))[0][1,temp-1,:]

tensor([True, True, True, True, True])

In [99]:
import torch.functional as F

class RNNModel(nn.Module):
    """The RNN model.

    Defined in :numref:`sec_rnn-concise`"""
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        # If the RNN is bidirectional (to be introduced later),
        # `num_directions` should be 2, else it should be 1.
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # The fully connected layer will first change the shape of `Y` to
        # (`num_steps` * `batch_size`, `num_hiddens`). Its output shape is
        # (`num_steps` * `batch_size`, `vocab_size`).
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # `nn.GRU` takes a tensor as hidden state
            return  torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),
                                device=device)
        else:
            # `nn.LSTM` takes a tuple of hidden states
            return (torch.zeros((
                self.num_directions * self.rnn.num_layers,
                batch_size, self.num_hiddens), device=device),
                    torch.zeros((
                        self.num_directions * self.rnn.num_layers,
                        batch_size, self.num_hiddens), device=device))


In [90]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        return (
            weight.new( batch_size,self.n_layers, self.hidden_dim)
            .zero_()
        )

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())

In [95]:
model=GRUNet(input_dim=5,hidden_dim=8,output_dim=5,n_layers=2,drop_prob=0)


In [98]:
next(iter(train_loader))[0][0].shape,model.init_hidden(BATCH_SIZE).shape

(torch.Size([48, 5]), torch.Size([32, 2, 8]))

In [91]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1E-5)

model(next(iter(train_loader)[0]),model.init_hidden(BATCH_SIZE))

# TRAIN_WITH_PROGRESS_BAR(MODEL=model,
#                         NUM_EPOCHS=1,
#                         OPTIMIZER=optimizer,
#                         TRAIN_LOADER=train_loader,
#                         TEST_LOADER=test_loader)

TypeError: '_SingleProcessDataLoaderIter' object is not subscriptable

In [89]:
model.init_hidden(BATCH_SIZE).shape

torch.Size([2, 32, 8])

In [25]:
import time

batch_size=BATCH_SIZE

def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=5, model_type="GRU",device=GET_DEVICE(0)):
    
    # Setting common hyperparameters
    input_dim = 5
    output_dim = 5
    n_layers = 2
    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)

    # Defining loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)

    model.train()
    print(f"Starting Training of {model_type} model")
    epoch_times = []
    # Start training loop
    for epoch in range(1,EPOCHS+1):
        start_time = time.clock()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()

            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            if counter%1 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.clock()
        print(
            f"Epoch {epoch}/{EPOCHS} Done, Total Loss: {avg_loss / len(train_loader)}"
        )
        print(f"Time Elapsed for Epoch: {str(current_time - start_time)} seconds")
        epoch_times.append(current_time-start_time)
    print(f"Total Training Time: {str(sum(epoch_times))} seconds")
    return model


In [26]:
lr = 0.001
gru_model = train(train_loader, lr, model_type="GRU")

Starting Training of GRU model


c:\Users\CTC\anaconda3\envs\d2l\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


ValueError: too many values to unpack (expected 2)